In [3]:
import torch

ModuleNotFoundError: No module named 'torch'

In [1]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

ModuleNotFoundError: No module named 'torch'

In [4]:
!ls drive/My\ Drive/Colab\ Notebooks/bert

 bert_cazzeggio.ipynb				  data
 BERT_Fine_Tuning_Sentence_Classification.ipynb   decode_strange_tweets.ipynb
'BERT Word Embeddings.ipynb'			  new_bert_cazzeggio.ipynb


In [6]:
!ls drive/My\ Drive/Colab\ Notebooks/bert/data

minitraining.tsv.gz
text_tokens_padded_50k.csv
tweet_features_text_tokens_padded.csv.gz
tweets_text_longer_than_280.csv
tweets_text_longer_than_bert_limit.csv
tweets_text_no_special_chars_50k.csv


In [ ]:
BERT_PATH = "drive/My Drive/Colab Notebooks/bert/data"
COLUMN_NAME = "raw_tweet_features_text_tokens"
PATH = BERT_PATH+"/text_tokens_padded_50k.csv"
RESULT_PATH = BERT_PATH+"/sentence_embeddings_50k.csv"
N_ROWS = 10000
CHUNKSIZE = 10
PAD = int(0)
MAX_LENGTH = 511
VOCAB = 'bert-base-multilingual-cased'

In [8]:
!pip install transformers

     |████████████████████████████████| 542kB 8.6MB/s eta 0:00:01
     |████████████████████████████████| 870kB 19.8MB/s 
     |████████████████████████████████| 1.0MB 47.6MB/s 
     |████████████████████████████████| 3.7MB 58.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=b16342ff63b72afee033c897c7c9301456e99cc3df13d97499e386214fb65da3
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [ ]:
import pandas as pd
import numpy as np

In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(VOCAB)

In [ ]:
vocab_size = len(tokenizer.vocab.keys())

In [12]:
vocab_size

119547

In [13]:
tokenizer.vocab['[PAD]']

0

## Create the model from a pre-trained one

In [145]:
from transformers import BertModel, BertForSequenceClassification, AdamW, BertConfig

# Load BERT model, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertModel.from_pretrained(
    VOCAB, # Use the 12-layer BERT model
)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [146]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 199 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                       (119547, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

==== First Transformer ====

encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                   

In [147]:
# Tell pytorch to run this model on the GPU.
model.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

## Load tweets from csv file as lists of int tokens

In [ ]:
tokenized_tweets = []

In [ ]:
def save_tweet(index, text):
    string = str(index) + ','
    for item in text:
        string += str(item) + '\t'
    result_file.write(string + '\n')

In [ ]:
f_to_int = lambda x: int(x)
f_int = lambda x: list(map(f_to_int, x.replace('\n','').split('\t')))

In [ ]:
result_file = open(RESULT_PATH, "w+")

In [226]:
result_file.write("tweet_features_tweet_id,sentence_embeddings\n")

44

In [ ]:
tokens_file = open(PATH, "r")

In [ ]:
# model.eval()  # is this needed ???

In [ ]:
%%time

# ~4 MINUTES EXECUTION ON THE WHOLE DATASET

# ignore header
tokens_file.readline()

finished = False
i = 0

while not finished:  # and i < 100:

    if i % 10000 == 0:
      print(i)
            
    line = str(tokens_file.readline())
    
    if line != '':
          line = line.split(',')[1]
          input_tokens = f_int(line)  # NOTE: bert wants lists

          #print(input_tokens)

          # create attention mask and convert both the mask
          #  and the input tokens to pytorch tensors
          mask = torch.tensor([[int(token_id > 0) for token_id in input_tokens]])
          input_tokens = torch.tensor([input_tokens])

          # move tensors to GPU
          input_tokens = input_tokens.to(device) 
          mask = mask.to(device) 

          outputs = model(input_ids=input_tokens, attention_mask=mask)

          #print(outputs)
          #print(outputs[0][0].size())
          #print(outputs[1][0].size())

          embeddings = outputs[1][0].tolist()

          save_tweet(i, embeddings)
        
    else:
        finished = True

    i += 1

0
10000
20000
30000


In [ ]:
tokens_file.close()

result_file.close()